<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Using-2MASS-Photometry" data-toc-modified-id="Using-2MASS-Photometry-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Using 2MASS Photometry</a></span></li><li><span><a href="#Using-GAIA-Measurements" data-toc-modified-id="Using-GAIA-Measurements-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Using GAIA Measurements</a></span></li></ul></div>

In [1]:
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.tools import make_subplots
import pandas as pd
from uncertainties import unumpy

In [2]:
main_dir = "/u/kaimibk/Documents/Research/NAC/"
data_dir = main_dir+"data/"
out_dir = main_dir+"output/"

In [3]:
df = pd.read_csv(data_dir+"Pleiades_master.csv", index_col=0)
df.head(2)

,Name,RA,Dec,K2MASS,(V-K)o,Period,Ref,BCK,logL/Lo,M/Mo,...,k_msigcom,radius_sigma,lum_sigma,teff_sigma,V_iso,V_iso_err,iso_rad,iso_rad_Err,iso_teff,iso_teff_Err
0,J03394633+2358531,54.943067,23.981442,11.83,4.89,1.138,1,2.73,-1.66,0.41,...,0.022,NaN,NaN,358.07666,18.646667,0.593949,0.419426,0.013360,4222.314643,100.0
1,J03414664+2301198,55.444367,23.022186,13.16,5.63,0.294,1,2.81,-2.23,0.19,...,0.028,NaN,NaN,NaN,46.221644,1.654124,0.268599,0.009612,3951.132391,100.0


### Using 2MASS Photometry

Here we inspect the color magnitude diagram using the *j*, *h*, and *k* magnitudes reported by the 2MASS survey. We investigate *k* vs. *j - k* AND *j* vs. *j - h*.

In [4]:
j = unumpy.uarray(df.j_m.values, df.j_msigcom.values)
h = unumpy.uarray(df.h_m.values, df.h_msigcom.values)
k = unumpy.uarray(df.k_m.values, df.k_msigcom.values)

In [5]:
one_to_one = np.arange(0, 1.4)

c1 = df.iso_rad < np.interp(df["R/Ro"], one_to_one, one_to_one)

df.Name[c1].head(4)

14    J03424239+2320218
22    J03430293+2440110
29    J03432662+2459395
30    J03432743+2237413
Name: Name, dtype: object

In [6]:
df.columns

Index(['Name', 'RA', 'Dec', 'K2MASS', '(V-K)o', 'Period', 'Ref', 'BCK',
       'logL/Lo', 'M/Mo', 'R/Ro', '(vsini)p', 'MK', 'SNR', 'RV', 'SRV', 'FWHM',
       'SFWHM', 'FWHMo', 'VSINI', 'EVSINI', 'Rsini', 'distribution',
       'parallax', 'parallax_error', 'radius_val', 'radius_percentile_lower',
       'radius_percentile_upper', 'lum_val', 'lum_percentile_lower',
       'lum_percentile_upper', 'teff_val', 'teff_percentile_lower',
       'teff_percentile_upper', 'bp_rp', 'phot_g_mean_mag', 'j_m', 'j_msigcom',
       'h_m', 'h_msigcom', 'k_m', 'k_msigcom', 'radius_sigma', 'lum_sigma',
       'teff_sigma', 'V_iso', 'V_iso_err', 'iso_rad', 'iso_rad_Err',
       'iso_teff', 'iso_teff_Err'],
      dtype='object')

In [7]:
text = [df.Name[i]+"\n Iso Radius: {} \n Jackson Radius {}".format(df.iso_rad[i], df["R/Ro"][i]) for i in range(len(df))]

In [8]:
text = np.array(text)

In [11]:
j_k_trace = go.Scatter(
    mode = "markers",
    text = text[~c1],
    hoverinfo = "text+x+y",
    
    x = unumpy.nominal_values(j-k)[~c1],
    y = unumpy.nominal_values(k)[~c1],
    
    error_x = {
        "type" : "data",
        "array" : unumpy.std_devs(j-k)[~c1],
        "visible" : False,
        "color" : 'black'
        },
    error_y = {
        "type" : "data",
        "array" : unumpy.std_devs(k)[~c1],
        "visible" : False,
        "color" : 'black'
        },
    marker = {
        #"color" : 'rgba(10, 100, 255, 0.3)',
        "size" : 5,
        "line" : {
            "color" : 'rgb(0, 0, 0)',
            "width" : 0.5
            }
        }
    )
j_k_trace2 = go.Scatter(
    mode = "markers",
    text = text[c1],
    hoverinfo = "text+x+y",
    
    x = unumpy.nominal_values(j-k)[c1],
    y = unumpy.nominal_values(k)[c1],
    
    error_x = {
        "type" : "data",
        "array" : unumpy.std_devs(j-k)[c1],
        "visible" : False,
        "color" : 'black'
        },
    error_y = {
        "type" : "data",
        "array" : unumpy.std_devs(k)[c1],
        "visible" : False,
        "color" : 'black'
        },
    marker = {
        #"color" : 'rgba(10, 100, 255, 0.3)',
        "size" : 5,
        "line" : {
            "color" : 'rgb(0, 0, 0)',
            "width" : 0.5
            }
        }
    )

j_h_trace = go.Scatter(
    mode = "markers",
    text = text[~c1],
    hoverinfo = "text+x+y",
    
    x = unumpy.nominal_values(j-h)[~c1],
    y = unumpy.nominal_values(h)[~c1],
    
    error_x = {
        "type" : "data",
        "array" : unumpy.std_devs(j-h)[~c1],
        "visible" : False,
        "color" : 'black'
        },
    error_y = {
        "type" : "data",
        "array" : unumpy.std_devs(j)[~c1],
        "visible" : False,
        "color" : 'black'
        },
    marker = {
        #"color" : 'rgba(255, 100, 15, 0.3)',
        "size" : 5,
        "line" : {
            "color" : 'rgb(0, 0, 0)',
            "width" : 0.5
            }
        }
    )

j_h_trace2 = go.Scatter(
    mode = "markers",
    text = text[c1],
    hoverinfo = "text+x+y",
    
    x = unumpy.nominal_values(j-h)[c1],
    y = unumpy.nominal_values(h)[c1],
    
    error_x = {
        "type" : "data",
        "array" : unumpy.std_devs(j-h)[c1],
        "visible" : False,
        "color" : 'black'
        },
    error_y = {
        "type" : "data",
        "array" : unumpy.std_devs(j)[c1],
        "visible" : False,
        "color" : 'black'
        },
    marker = {
        #"color" : 'rgba(255, 100, 15, 0.3)',
        "size" : 5,
        "line" : {
            "color" : 'rgb(0, 0, 0)',
            "width" : 0.5
            }
        }
    )

fig = make_subplots(rows=1, cols=2)
fig.append_trace(j_k_trace, 1, 1)
fig.append_trace(j_k_trace2, 1, 1)
fig.append_trace(j_h_trace, 1, 2)
fig.append_trace(j_h_trace2, 1, 2)

# fig = make_subplots(rows=2, cols=1)
# fig.append_trace(j_k_trace, 1, 1)
# fig.append_trace(j_k_trace2, 1, 1)
# fig.append_trace(j_h_trace, 2, 1)
# fig.append_trace(j_h_trace2, 2, 1)

fig['layout'].update(showlegend=False)

fig['layout']['xaxis1'].update(title='j-k', mirror='ticks', linecolor='black', linewidth=1)
fig['layout']['xaxis2'].update(title='j-h', mirror='ticks', linecolor='black', linewidth=1)

fig['layout']['yaxis1'].update(title='k', autorange = 'reversed', mirror='ticks', linecolor='black', linewidth=1)
fig['layout']['yaxis2'].update(title='j', autorange = 'reversed', mirror='ticks', linecolor='black', linewidth=1)



#py.iplot(fig, filename="cmd_2mass_jk")

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]



### Using GAIA Measurements

Here we inspect the color magnitude diagram using information provided by the GAIA survey, specifically *bp_rp*, color, and *phot_g_mean_mag*, apparent magnitude. Recall that apparent magnitude is converted using the following relation,

$$M = m + 5(\log_{10} p + 1)$$

In [12]:
df["gaia_abs_mag"] = df.phot_g_mean_mag + 5*np.log10(df.parallax + 1)

In [13]:
df.gaia_abs_mag.head(5)

0    20.176881
1    21.836556
2    22.282054
3    21.659404
4    20.259029
Name: gaia_abs_mag, dtype: float64

In [14]:
gaia_trace = go.Scatter(
    mode = "markers",
    text = text[~c1],
    hoverinfo = "text+x+y",
    
    x = df.bp_rp[~c1],
    y = df.gaia_abs_mag[~c1],

    marker = {
        "size" : 6,
        "line" : {
            "color" : 'rgb(0, 0, 0)',
            "width" : 1
            }
        }
    )

gaia_trace2 = go.Scatter(
    mode = "markers",
    text = text[c1],
    hoverinfo = "text+x+y",
    
    x = df.bp_rp[c1],
    y = df.gaia_abs_mag[c1],

    marker = {
        "size" : 6,
        "line" : {
            "color" : 'rgb(0, 0, 0)',
            "width" : 1
            }
        }
    )

layout = go.Layout(
            xaxis={
                "title" : "bp_rp",
                "mirror" : "ticks",
                "linecolor" : "black",
                "linewidth" : 1
            },
            yaxis={
                "title" : "M_G",
                "mirror" : "ticks",
                "linecolor" : "black",
                "linewidth" : 1,
                "autorange" : "reversed"
            },
    )

#figure = go.Figure(data=[gaia_trace, gaia_trace2], layout=layout)

#py.iplot(figure, filename="cmd_gaia")

In [15]:
fig = make_subplots(rows=2, cols=2, specs=[[{}, {}], [{'colspan': 2}, None]],
                          subplot_titles=('2MASS J-K','2MASS J-H', 'GAIA G Magnitude'))

fig.append_trace(j_k_trace, 1, 1)
fig.append_trace(j_k_trace2, 1, 1)

fig.append_trace(j_h_trace, 1, 2)
fig.append_trace(j_h_trace2, 1, 2)

fig.append_trace(gaia_trace, 2, 1)
fig.append_trace(gaia_trace2, 2, 1)

fig['layout'].update(showlegend=False)

fig['layout']['xaxis1'].update(title='j-k', mirror='ticks', linecolor='black', linewidth=1)
fig['layout']['xaxis2'].update(title='j-h', mirror='ticks', linecolor='black', linewidth=1)
fig['layout']['xaxis3'].update(title='bp_rp', mirror='ticks', linecolor='black', linewidth=1)


fig['layout']['yaxis1'].update(title='k', autorange = 'reversed', mirror='ticks', linecolor='black', linewidth=1)
fig['layout']['yaxis2'].update(title='j', autorange = 'reversed', mirror='ticks', linecolor='black', linewidth=1)
fig['layout']['yaxis3'].update(title='M_G', autorange = 'reversed', mirror='ticks', linecolor='black', linewidth=1)


py.iplot(fig, filename='cmd_combined')

/u/kaimibk/miniconda3/lib/python3.6/site-packages/plotly/graph_objs/_deprecations.py:318: DeprecationWarning:

plotly.graph_objs.Font is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Font
  - plotly.graph_objs.layout.hoverlabel.Font
  - etc.




This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]
[ (2,1) x3,y3           -      ]



In [16]:
df[df.iso_rad == 0].drop(columns="Name.1").to_csv(main_dir+"iso_fail.csv")

KeyError: "labels ['Name.1'] not contained in axis"